# Sección 2: Preparación de Datos y Feature Engineering

## Contexto
El EDA (notebook 01) identificó:
- Desbalance 70/30 (no-churn/churn)
- Multicolinealidad entre ads y offline_listening
- Correlaciones débiles con target → modelos de árbol son mejores que lineales

## Flujo
Este notebook genera 3 versiones de datos para comparación:

1. **Baseline**: 21 features, sin FE, sin SMOTE, class_weight='balanced'
2. **FE**: 36 features (21 base + 15 engineered), sin SMOTE
3. **FE+SMOTE**: 36 features, SMOTE aplicado solo a train (50/50 balance)

Todas con split 60/20/20 estratificado (train/val/test).

## Features Engineered (15 nuevos)
- **Interacciones** (3): skip_ads_interaction, usage_intensity, offline_engagement
- **Ratios** (2): listening_efficiency, ads_per_song
- **No-lineales** (2): skip_rate_squared, ads_squared
- **Scores** (2): engagement_score, churn_risk_score
- **Binning** (2): age_group (cuartiles), listening_group (tertiles)
- **GMM** (3): gmm_cluster, gmm_prob_cluster_0/1 (se omite la 3ra por redundancia)

Total: 21 base + 14 engineered = 35 features finales

## Notas Técnicas
- StandardScaler ajustado en train, aplicado a val/test (sin data leakage)
- Bins calculados en train, aplicados a val/test
- GMM entrenado solo en train, usado para predict en val/test
- SMOTE aplicado SOLO en train (val/test mantienen distribución original 70/30)

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
from IPython.display import display, Markdown

warnings.filterwarnings('ignore')

# Helper
def print_section(title):
    display(Markdown(f"### {title}"))
    display(Markdown('---'))

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

## 2.1 Cargar y Explorar Datos Crudos

In [2]:
# Cargar con pandas
df = pd.read_parquet('data/spotify_churn.parquet')

print(f"Shape: {df.shape}")
print(f"Dtypes:\n{df.dtypes}")

# Separar variables por tipo (según EDA en notebook 01)
# Numéricas continuas: age, listening_time, songs_played_per_day, skip_rate, ads_listened_per_week
# Binarias: offline_listening (0/1)
# Categóricas: gender, country, subscription_type, device_type
# Target: is_churned (binaria)

numerical_continuous = ['age', 'listening_time', 'songs_played_per_day', 'skip_rate', 'ads_listened_per_week']
numerical_binary = ['offline_listening']
categorical_cols = df.select_dtypes(include=['object', 'string']).columns.tolist()
target = 'is_churned'

print(f"\nNuméricas continuas: {len(numerical_continuous)}")
print(f"  {numerical_continuous}")
print(f"\nNuméricas binarias: {len(numerical_binary)}")
print(f"  {numerical_binary}")
print(f"\nCategóricas: {len(categorical_cols)}")
print(f"  {categorical_cols}")
print(f"\nTarget: {target}")

Shape: (8000, 12)
Dtypes:
user_id                    int64
gender                    object
age                        int64
country                   object
subscription_type         object
listening_time             int64
songs_played_per_day       int64
skip_rate                float64
device_type               object
ads_listened_per_week      int64
offline_listening          int64
is_churned                 int64
dtype: object

Numéricas continuas: 5
  ['age', 'listening_time', 'songs_played_per_day', 'skip_rate', 'ads_listened_per_week']

Numéricas binarias: 1
  ['offline_listening']

Categóricas: 4
  ['gender', 'country', 'subscription_type', 'device_type']

Target: is_churned


## 2.2 Encoding y Normalización

### 2.2.1 One-Hot Encoding para Categóricas

In [3]:
print_section("Encoding: Categóricas → One-Hot")

# One-hot encoding para categóricas
X_cat = pd.get_dummies(df[categorical_cols], drop_first=True, dtype=int)

# Numéricas (continuas + binarias) sin cambios
numerical_cols = numerical_continuous + numerical_binary
X_num = df[numerical_cols]

# Combinar
X = pd.concat([X_num, X_cat], axis=1)
y = df[target].astype(int)

# Resumen
display(Markdown(f"**Features totales:** {X.shape[1]}  \\n**Muestra:** {X.shape[0]}"))
display(Markdown(f"**Composición:** {len(numerical_continuous)} continuas + {len(numerical_binary)} binarias + {X_cat.shape[1]} categóricas codificadas"))

display(Markdown("**Primeras 3 filas de X:**\n"))
display(X.head(3))

### Encoding: Categóricas → One-Hot

---

**Features totales:** 20  \n**Muestra:** 8000

**Composición:** 5 continuas + 1 binarias + 14 categóricas codificadas

**Primeras 3 filas de X:**


,age,listening_time,songs_played_per_day,skip_rate,ads_listened_per_week,offline_listening,gender_Male,gender_Other,country_CA,country_DE,country_FR,country_IN,country_PK,country_UK,country_US,subscription_type_Free,subscription_type_Premium,subscription_type_Student,device_type_Mobile,device_type_Web
0,54,26,23,0.20,31,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
1,33,141,62,0.34,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1
2,38,199,38,0.04,0,1,1,0,0,0,0,0,0,0,0,0,1,0,1,0


### 2.2.2 Normalización con StandardScaler

In [4]:
print_section("Normalizar Numéricas Continuas con StandardScaler")

scaler = StandardScaler()

# Aplicar scaler SOLO a numéricas continuas (NO a binarias)
X_scaled = X.copy()
X_scaled[numerical_continuous] = scaler.fit_transform(X[numerical_continuous])

display(Markdown("**Escala aplicada solo a variables continuas** (binarias se mantienen sin cambios)."))

display(Markdown("**Antes del scaling (continuas):**"))
display(X[numerical_continuous].describe())

display(Markdown("**Después del scaling:**"))
display(X_scaled[numerical_continuous].describe())

### Normalizar Numéricas Continuas con StandardScaler

---

**Escala aplicada solo a variables continuas** (binarias se mantienen sin cambios).

**Antes del scaling (continuas):**

,age,listening_time,songs_played_per_day,skip_rate,ads_listened_per_week
count,8000.000000,8000.000000,8000.000000,8000.000000,8000.000000
mean,37.662125,154.068250,50.127250,0.300127,6.943875
std,12.740359,84.015596,28.449762,0.173594,13.617953
min,16.000000,10.000000,1.000000,0.000000,0.000000
25%,26.000000,81.000000,25.000000,0.150000,0.000000
50%,38.000000,154.000000,50.000000,0.300000,0.000000
75%,49.000000,227.000000,75.000000,0.450000,5.000000
max,59.000000,299.000000,99.000000,0.600000,49.000000


**Después del scaling:**

,age,listening_time,songs_played_per_day,skip_rate,ads_listened_per_week
count,8.000000e+03,8.000000e+03,8.000000e+03,8.000000e+03,8.000000e+03
mean,-2.502443e-16,-6.883383e-17,1.172396e-16,3.748113e-16,-8.881784e-19
std,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00,1.000063e+00
min,-1.700382e+00,-1.714887e+00,-1.726915e+00,-1.729011e+00,-5.099378e-01
25%,-9.154258e-01,-8.697530e-01,-8.832700e-01,-8.648730e-01,-5.099378e-01
50%,2.652171e-02,-8.123999e-04,-4.473076e-03,-7.345177e-04,-5.099378e-01
75%,8.899736e-01,8.681282e-01,8.743238e-01,8.634040e-01,-1.427525e-01
max,1.674930e+00,1.725166e+00,1.717969e+00,1.727542e+00,3.088478e+00


## 2.3 Split Train/Val/Test Estratificado

In [5]:
print_section("Split: 60% Train, 20% Val, 20% Test (Estratificado)")

X_work = X_scaled.copy()

# Train+Val (80%) vs Test (20%)
X_temp, X_test, y_temp, y_test = train_test_split(
    X_work, y, test_size=0.20, random_state=42, stratify=y
)

# Split 2: Train (75% de temp = 60% total) vs Val (25% de temp = 20% total)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp
)

# Resumen de tamaños
split_stats = pd.DataFrame({
    'split': ['Train', 'Val', 'Test'],
    'n': [X_train.shape[0], X_val.shape[0], X_test.shape[0]],
    'pct': [X_train.shape[0]/X_work.shape[0]*100, X_val.shape[0]/X_work.shape[0]*100, X_test.shape[0]/X_work.shape[0]*100]
})
display(Markdown("**Tamaños por split:**"))
display(split_stats)

# Distribución de clases por split
def class_dist(series):
    return pd.Series({
        'churn_count': int((series == 1).sum()),
        'total': int(len(series)),
        'pct_churn': float((series == 1).mean()*100)
    })

class_df = pd.DataFrame([class_dist(y_train), class_dist(y_val), class_dist(y_test)], index=['Train','Val','Test'])
display(Markdown("**Distribución de clases por split:**"))
display(class_df)

display(Markdown("**Stratificación:** todas las particiones tienen ~30% churn (ver tabla) ✓"))

### Split: 60% Train, 20% Val, 20% Test (Estratificado)

---

**Tamaños por split:**

,split,n,pct
0,Train,4800,60.0
1,Val,1600,20.0
2,Test,1600,20.0


**Distribución de clases por split:**

,churn_count,total,pct_churn
Train,1243.0,4800.0,25.895833
Val,414.0,1600.0,25.875000
Test,414.0,1600.0,25.875000


**Stratificación:** todas las particiones tienen ~30% churn (ver tabla) ✓

## 2.4 Exportar Baseline sin FE

In [6]:
print_section("Exportar Baseline: Datos Preprocesados (SIN FE)")

# Convertir a dict con numpy arrays
train_data = {
    'X': X_train.values,
    'y': y_train.values,
    'columns': X_train.columns.tolist()
}

val_data = {
    'X': X_val.values,
    'y': y_val.values,
    'columns': X_val.columns.tolist()
}

test_data = {
    'X': X_test.values,
    'y': y_test.values,
    'columns': X_test.columns.tolist()
}

with open('data/train_data.pkl', 'wb') as f:
    pickle.dump(train_data, f)

with open('data/val_data.pkl', 'wb') as f:
    pickle.dump(val_data, f)

with open('data/test_data.pkl', 'wb') as f:
    pickle.dump(test_data, f)

feature_names_base = X_train.columns.tolist()
display(Markdown(f"**✅ Baseline guardado (21 features):**"))
display(Markdown("- `data/train_data.pkl`"))
display(Markdown("- `data/val_data.pkl`"))
display(Markdown("- `data/test_data.pkl`"))

### Exportar Baseline: Datos Preprocesados (SIN FE)

---

**✅ Baseline guardado (21 features):**

- `data/train_data.pkl`

- `data/val_data.pkl`

- `data/test_data.pkl`

## 2.5 Feature Engineering

### 2.5.1 Crear Features 

In [7]:

# Convertir a DataFrame para feature engineering
df_train = pd.DataFrame(X_train.values, columns=feature_names_base)
df_train['is_churned'] = y_train.values

df_val = pd.DataFrame(X_val.values, columns=feature_names_base)
df_val['is_churned'] = y_val.values

df_test = pd.DataFrame(X_test.values, columns=feature_names_base)
df_test['is_churned'] = y_test.values


#### 2.Interacciones: Combinaciones que capturan efectos conjuntos

- **skip_ads_interaction**: Usuario con muchos skips Y muchos ads simultáneamente → señal fuerte de insatisfacción
- **usage_intensity**: Escucha mucho Y durante mucho tiempo → engagement genuino
- **offline_engagement**: Escucha sin conexión Y consume muchas canciones → patrón móvil, usuario leal

#### Ratios: Magnitudes relativas que normalizan el consumo

- **listening_efficiency**: Velocidad de consumo (canciones/tiempo); busca distinguir entre usuarios que buscan canciones nuevas y los que escuchan por más tiempo
- **ads_per_song**: Tolerancia relativa a publicidad; ratio bajo = usuario tolera bien ads, ratio alto = molesto con ads

#### Transformaciones No-Lineales: Efectos de segundo orden

- **skip_rate_squared**: Amplifica el efecto de skip; usuario con 0.9 → 0.81 es mucho más "peligroso" que uno con 0.5
- **ads_squared**: Fatiga publicitaria exponencial; detecta usuarios donde el exceso de anuncios se acumula

#### Scores Agregados: Métricas holísticas de usuario

- **engagement_score**: Suma factores positivos (canciones, tiempo, offline) y resta negativos (skips, ads×0.5); valores altos = usuario sano
- **churn_risk_score**: Indicador inverso; valores altos = usuario en riesgo (se queja más de lo que consume)

In [8]:
def create_engineered_features(df):

    df_fe = df.copy()

    # Interacciones
    df_fe['skip_ads_interaction'] = df_fe['skip_rate'] * df_fe['ads_listened_per_week']
    df_fe['usage_intensity'] = df_fe['songs_played_per_day'] * df_fe['listening_time']
    df_fe['offline_engagement'] = df_fe['offline_listening'] * df_fe['songs_played_per_day']

    # Ratios
    df_fe['listening_efficiency'] = df_fe['songs_played_per_day'] / (df_fe['listening_time'] + 0.01)
    df_fe['ads_per_song'] = df_fe['ads_listened_per_week'] / (df_fe['songs_played_per_day'] + 0.01)

    # Transformaciones no-lineales
    df_fe['skip_rate_squared'] = df_fe['skip_rate'] ** 2
    df_fe['ads_squared'] = df_fe['ads_listened_per_week'] ** 2

    # Scores agregados
    df_fe['engagement_score'] = (
        df_fe['songs_played_per_day'] + df_fe['listening_time'] + df_fe['offline_listening']
        - df_fe['skip_rate'] - (df_fe['ads_listened_per_week'] * 0.5)
    )

    df_fe['churn_risk_score'] = (
        df_fe['skip_rate'] + df_fe['ads_listened_per_week']
        - df_fe['songs_played_per_day'] - df_fe['listening_time']
    )

    return df_fe

In [9]:

# Aplicar features básicos
df_train_fe = create_engineered_features(df_train)
df_val_fe = create_engineered_features(df_val)
df_test_fe = create_engineered_features(df_test)


### 2.5.2 Crear Binning y GMM Clustering

#### Binning: Discretización en grupos para capturar no-linealidades

- **age_group**: Cuartiles de edad; jóvenes vs adultos tienen patrones de consumo y churn muy distintos
- **listening_group**: Tertiles de tiempo de escucha; light/medium/heavy listeners tienen comportamientos completamente diferentes

#### GMM Clustering: Segmentación sin etiquetas

- **gmm_cluster**: Asignación a 3 clusters (bajo/medio/alto engagement) sin supervisión; captura estructura natural de usuarios
- **gmm_prob_cluster_0/1/2**: Probabilidades suaves de pertenencia; más flexible que etiquetas duras, el modelo puede aprender con la incertidumbre

In [10]:

# Binning: calcular bins en train y aplicarlos a val/test
age_bins = [-np.inf,
            df_train_fe['age'].quantile(0.25),
            df_train_fe['age'].quantile(0.5),
            df_train_fe['age'].quantile(0.75),
            np.inf]

time_bins = [-np.inf,
             df_train_fe['listening_time'].quantile(0.33),
             df_train_fe['listening_time'].quantile(0.67),
             np.inf]

for df in [df_train_fe, df_val_fe, df_test_fe]:
    df['age_group'] = pd.cut(df['age'], bins=age_bins, labels=[0, 1, 2, 3]).astype(float)
    df['listening_group'] = pd.cut(df['listening_time'], bins=time_bins, labels=[0, 1, 2]).astype(float)

# GMM clustering: entrenar solo en train y aplicar a los demás conjuntos
cluster_features = ['songs_played_per_day', 'listening_time', 'skip_rate',
                   'ads_listened_per_week', 'offline_listening']
X_train_cluster = df_train_fe[cluster_features].values

# reg_covar agrega regularización L2 a la covarianza (estabilidad ante multicolinealidad)
gmm_model = GaussianMixture(n_components=3, random_state=42, covariance_type='full', reg_covar=1e-5)
gmm_model.fit(X_train_cluster)

for df in [df_train_fe, df_val_fe, df_test_fe]:
    X_cluster = df[cluster_features].values
    df['gmm_cluster'] = gmm_model.predict(X_cluster)
    cluster_probs = gmm_model.predict_proba(X_cluster)
    df['gmm_prob_cluster_0'] = cluster_probs[:, 0]
    df['gmm_prob_cluster_1'] = cluster_probs[:, 1]
    # podemos omitir gmm_prob_cluster_2 para evitar multicolinealidad perfecta (suma=1) 

### 2.5.3 Separación de Features y Target

In [11]:
# Escalar todos los features finales 
#Esto normaliza los nuevos features (scores, ratios, GMM) y re-ajusta los base para evitar data leakage
scaler_final = StandardScaler()
cols_features = [c for c in df_train_fe.columns if c != 'is_churned']

df_train_fe[cols_features] = scaler_final.fit_transform(df_train_fe[cols_features])
df_val_fe[cols_features] = scaler_final.transform(df_val_fe[cols_features])
df_test_fe[cols_features] = scaler_final.transform(df_test_fe[cols_features])

# Separar X, y
X_train_fe = df_train_fe.drop('is_churned', axis=1).values
y_train_fe = df_train_fe['is_churned'].values

X_val_fe = df_val_fe.drop('is_churned', axis=1).values
y_val_fe = df_val_fe['is_churned'].values

X_test_fe = df_test_fe.drop('is_churned', axis=1).values
y_test_fe = df_test_fe['is_churned'].values

# Nombres de columnas finales
feature_names_fe = df_train_fe.drop('is_churned', axis=1).columns.tolist()

# Contar features nuevos
original_features = set(feature_names_base)
engineered_count = len([f for f in feature_names_fe if f not in original_features])

display(Markdown(f"**Train shape:** X={X_train_fe.shape}, y={y_train_fe.shape}"))
display(Markdown(f"**Val shape:** X={X_val_fe.shape}, y={y_val_fe.shape}"))
display(Markdown(f"**Test shape:** X={X_test_fe.shape}, y={y_test_fe.shape}"))
display(Markdown(f"**Features originales:** {len(original_features)}"))
display(Markdown(f"**Features engineered:** {engineered_count}"))
display(Markdown(f"**Total features:** {len(feature_names_fe)}"))

**Train shape:** X=(4800, 34), y=(4800,)

**Val shape:** X=(1600, 34), y=(1600,)

**Test shape:** X=(1600, 34), y=(1600,)

**Features originales:** 20

**Features engineered:** 14

**Total features:** 34

## 2.6 Exportar Versiones: Con y Sin SMOTE

### 2.6.1 Exportar SIN SMOTE

In [15]:
print_section("Exportar Versión SIN SMOTE")

# Guardar train/val/test con feature engineering (sin SMOTE)
with open('data/train_data_fe.pkl', 'wb') as f:
    pickle.dump({'X': X_train_fe, 'y': y_train_fe, 'columns': feature_names_fe}, f)

with open('data/val_data_fe.pkl', 'wb') as f:
    pickle.dump({'X': X_val_fe, 'y': y_val_fe, 'columns': feature_names_fe}, f)

with open('data/test_data_fe.pkl', 'wb') as f:
    pickle.dump({'X': X_test_fe, 'y': y_test_fe, 'columns': feature_names_fe}, f)

display(Markdown("**Archivos guardados (SIN SMOTE):**"))
display(Markdown("- `data/train_data_fe.pkl`"))
display(Markdown("- `data/val_data_fe.pkl`"))
display(Markdown("- `data/test_data_fe.pkl`"))

### Exportar Versión SIN SMOTE

---

**Archivos guardados (SIN SMOTE):**

- `data/train_data_fe.pkl`

- `data/val_data_fe.pkl`

- `data/test_data_fe.pkl`

### 2.6.2 Aplicar SMOTE al Entrenamiento

In [16]:
print_section("Aplicar SMOTE al Conjunto de Entrenamiento")

# Aplicar SMOTE solo a train
smote = SMOTE(random_state=RANDOM_STATE)
X_train_fe_smote, y_train_fe_smote = smote.fit_resample(X_train_fe, y_train_fe)

# Distribución antes/después
balance_comparison = pd.DataFrame({
    'Dataset': ['Train (original)', 'Train (SMOTE)', 'Val', 'Test'],
    'No Churn': [
        (y_train_fe == 0).sum(),
        (y_train_fe_smote == 0).sum(),
        (y_val_fe == 0).sum(),
        (y_test_fe == 0).sum()
    ],
    'Churn': [
        (y_train_fe == 1).sum(),
        (y_train_fe_smote == 1).sum(),
        (y_val_fe == 1).sum(),
        (y_test_fe == 1).sum()
    ]
})

balance_comparison['Ratio'] = balance_comparison.apply(
    lambda row: f"{row['No Churn'] / row['Churn']:.2f}:1" if row['Churn'] > 0 else 'N/A', axis=1
)

display(balance_comparison)
display(Markdown(f"\n** SMOTE aplicado:** {len(y_train_fe)} → {len(y_train_fe_smote)} samples"))

### Aplicar SMOTE al Conjunto de Entrenamiento

---

,Dataset,No Churn,Churn,Ratio
0,Train (original),3557,1243,2.86:1
1,Train (SMOTE),3557,3557,1.00:1
2,Val,1186,414,2.86:1
3,Test,1186,414,2.86:1



** SMOTE aplicado:** 4800 → 7114 samples

### 2.6.3 Exportar CON SMOTE

In [17]:
print_section("Exportar Versión CON SMOTE")

# Guardar train con SMOTE (val/test sin SMOTE)
with open('data/train_data_fe_smote.pkl', 'wb') as f:
    pickle.dump({'X': X_train_fe_smote, 'y': y_train_fe_smote, 'columns': feature_names_fe}, f)

display(Markdown("** Archivos guardados (CON SMOTE):**"))
display(Markdown("- `data/train_data_fe_smote.pkl` (balanceado con SMOTE)"))
display(Markdown("- `data/val_data_fe.pkl` (sin SMOTE - reutilizado)"))
display(Markdown("- `data/test_data_fe.pkl` (sin SMOTE - reutilizado)"))

### Exportar Versión CON SMOTE

---

** Archivos guardados (CON SMOTE):**

- `data/train_data_fe_smote.pkl` (balanceado con SMOTE)

- `data/val_data_fe.pkl` (sin SMOTE - reutilizado)

- `data/test_data_fe.pkl` (sin SMOTE - reutilizado)